# WildNav with RoMa + DINOv2 on GPU

This notebook runs the WildNav localization pipeline using **RoMa** (CVPR 2024) with DINOv2 backbone for robust dense feature matching on Google Colab's GPU.

**Before running:** Make sure to enable GPU in Runtime > Change runtime type > Hardware accelerator > GPU (T4)

## 1. Check GPU Availability

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ WARNING: GPU not available! Enable it in Runtime > Change runtime type")

## 2. Clone Repository from GitHub

In [ ]:
!git clone https://github.com/SiddharthSirohi/betterwildnav.git
%cd betterwildnav

## 3. Install Dependencies

In [ ]:
# Install PyTorch with CUDA support (if not already installed)
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install project dependencies
!pip install -q numpy opencv-python-headless matplotlib pandas scikit-learn seaborn haversine requests Pillow

# Install RoMa (includes DINOv2 automatically)
!pip install -q romatch

print("✅ All dependencies installed!")
print("RoMa uses DINOv2 backbone natively - no separate installation needed!")

## 4. Verify Installation

In [ ]:
import sys
sys.path.append('/content/betterwildnav/wildnav/src')

# Test RoMa import
try:
    from romatch import roma_outdoor
    print("✅ RoMa imported successfully")
    
    # Test model loading
    import torch
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    roma_model = roma_outdoor(device=device)
    print(f"✅ RoMa model loaded successfully on {device}")
    print("   RoMa uses DINOv2 backbone automatically!")
    del roma_model  # Free memory
except Exception as e:
    print(f"✗ RoMa import/load failed: {e}")

## 5. Upload Your Data (Optional)

If you want to test with your own drone images and satellite maps, upload them here.
Otherwise, the repository should already contain sample data.

In [ ]:
# Uncomment to upload files
# from google.colab import files
# uploaded = files.upload()

# Check if data exists
import os
data_path = '/content/betterwildnav/wildnav/assets'
if os.path.exists(data_path):
    print(f"✅ Data directory found: {data_path}")
    print(f"   Query images: {len(os.listdir(data_path + '/query'))} files")
    print(f"   Map images: {len(os.listdir(data_path + '/map'))} files")
else:
    print("⚠️ WARNING: Data directory not found!")
    print("You may need to upload your data.")

## 6. Run RoMa Feature Matching Test

Quick test to verify RoMa works with GPU acceleration and DINOv2 backbone.

In [ ]:
%cd /content/betterwildnav/wildnav/src

# Run standalone RoMa test
!python test_roma_standalone.py

## 7. Run Full Localization Pipeline

This runs the complete WildNav pipeline with DINOv2 + LightGlue.

In [ ]:
%cd /content/betterwildnav/wildnav/src

import time
start_time = time.time()

# Run the main pipeline
!python wildnav.py

elapsed = time.time() - start_time
print(f"\n\n{'='*60}")
print(f"Total processing time: {elapsed/60:.2f} minutes")
print(f"{'='*60}")

## 8. View Results

In [ ]:
import pandas as pd
from IPython.display import Image, display
import os

# Display results CSV
results_path = '/content/betterwildnav/wildnav/results/calculated_coordinates.csv'
if os.path.exists(results_path):
    print("📊 Localization Results:\n")
    df = pd.read_csv(results_path, header=None)
    df.columns = ['Image', 'True_Lat', 'True_Lon', 'Calc_Lat', 'Calc_Lon', 
                  'Error_Lat', 'Error_Lon', 'Error_m', 'Col8', 'Col9', 'Col10']
    print(df[['Image', 'Error_m']].to_string(index=False))
    print(f"\n📍 Mean Absolute Error: {df['Error_m'].mean():.2f} meters")
else:
    print("⚠️ Results file not found")

# Display visualizations
print("\n🖼️ Localization Visualizations:\n")
results_dir = '/content/betterwildnav/wildnav/results'
if os.path.exists(results_dir):
    for img_file in sorted(os.listdir(results_dir)):
        if img_file.endswith('_located.png'):
            print(f"\n{img_file}:")
            display(Image(filename=os.path.join(results_dir, img_file), width=800))

## 9. Download Results

In [ ]:
from google.colab import files
import shutil

# Create a zip of results
!cd /content/betterwildnav/wildnav && zip -r results.zip results/

# Download
files.download('/content/betterwildnav/wildnav/results.zip')
print("✅ Results downloaded!")

## Performance Notes

**RoMa with DINOv2 Backbone:**
- **GPU (T4)**: ~1-3 seconds per image pair for dense matching
- **CPU**: ~30-60 seconds per image pair
- **Speedup**: 10-30x faster on GPU

**Why RoMa + DINOv2?**
- RoMa (CVPR 2024) is specifically designed to use DINOv2 as its backbone
- Provides dense, robust feature matching across large viewpoint changes
- Better generalization to outdoor drone-to-satellite matching scenarios

For production use, GPU is **strongly recommended** for RoMa-based localization.